In [1]:
sqlContext = SQLContext(sc)

## Load house prices

In [2]:
text_RDD = sc.textFile("/data/houses.txt")

In [3]:
def mapper_parse_lines(line):
    """Parse line into (neighborhoood, price) pair"""
    words = line.split()
    return (words[1], float(words[2]), int(words[0]))

In [4]:
house_prices_RDD = text_RDD.map(mapper_parse_lines)

In [5]:
house_prices_RDD.collect()

[(u'Downtown', 400000.0, 3),
 (u'Downtown', 240000.0, 2),
 (u'Hilltop', 650000.0, 3)]

In [6]:
house_prices_df = sqlContext.createDataFrame(house_prices_RDD,
                ["neighborhood", "price", "bedrooms"])

In [7]:
house_prices_df.show()

+------------+--------+--------+
|neighborhood|   price|bedrooms|
+------------+--------+--------+
|    Downtown|400000.0|       3|
|    Downtown|240000.0|       2|
|     Hilltop|650000.0|       3|
+------------+--------+--------+



In [8]:
house_prices_df.printSchema()

root
 |-- neighborhood: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: long (nullable = true)



## Load inflation

In [9]:
inflation_text_RDD = sc.textFile("/data/inflation.txt")

In [10]:
def mapper_parse__inflation_lines(line):
    """Parse line into (neighborhoood, inflation) pair"""
    words = line.split()
    return (words[0], float(words[1]))

In [11]:
inflation_RDD = inflation_text_RDD.map(mapper_parse__inflation_lines)

In [12]:
inflation_RDD.collect()

[(u'Downtown', 2.1), (u'Hilltop', 4.5)]

In [13]:
inflation_df = sqlContext.createDataFrame(inflation_RDD,
                        ["neighborhood", "inflation"])

In [14]:
inflation_df.printSchema()

root
 |-- neighborhood: string (nullable = true)
 |-- inflation: double (nullable = true)



## join

In [15]:
house_prices_df.join(inflation_df, "neighborhood").show()

+------------+--------+--------+---------+
|neighborhood|   price|bedrooms|inflation|
+------------+--------+--------+---------+
|    Downtown|400000.0|       3|      2.1|
|    Downtown|240000.0|       2|      2.1|
|     Hilltop|650000.0|       3|      4.5|
+------------+--------+--------+---------+



In [16]:
house_prices_with_inflation_df = house_prices_df.join(inflation_df, "neighborhood")

In [17]:
house_prices_nextyear_df = \
house_prices_with_inflation_df.withColumn("new_price",                                      
(house_prices_with_inflation_df.price * (1 +
        house_prices_with_inflation_df.inflation/100.)))

In [18]:
house_prices_nextyear_df.show()

+------------+--------+--------+---------+------------------+
|neighborhood|   price|bedrooms|inflation|         new_price|
+------------+--------+--------+---------+------------------+
|    Downtown|400000.0|       3|      2.1|408399.99999999994|
|    Downtown|240000.0|       2|      2.1|245039.99999999997|
|     Hilltop|650000.0|       3|      4.5|          679250.0|
+------------+--------+--------+---------+------------------+



In [19]:
house_prices_nextyear_df = house_prices_with_inflation_df.withColumn("new_price",                                      
    (house_prices_with_inflation_df.price * (1 +\
    house_prices_with_inflation_df.inflation/100.)).cast("int"))

In [20]:
house_prices_nextyear_df.show()

+------------+--------+--------+---------+---------+
|neighborhood|   price|bedrooms|inflation|new_price|
+------------+--------+--------+---------+---------+
|    Downtown|400000.0|       3|      2.1|   408399|
|    Downtown|240000.0|       2|      2.1|   245039|
|     Hilltop|650000.0|       3|      4.5|   679250|
+------------+--------+--------+---------+---------+



## reduce

In [21]:
house_prices_nextyear_df.groupBy("neighborhood").sum("new_price").show()

+------------+--------------+
|neighborhood|sum(new_price)|
+------------+--------------+
|    Downtown|        653438|
|     Hilltop|        679250|
+------------+--------------+



## Exercise

Find the **mean** price for next year by number of bedrooms

In [ ]:
# %load solution_house_price_join_dataframes.py
house_prices_nextyear_df.groupBy("bedrooms").mean("new_price").show()